# Artifact Rejector - FAIL #1

In [1]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

import hypnogram as hp
import kd_analysis.main.utils as kd
import kd_analysis.main.plots as kp
import neurodsp.plts.utils as dspu
import sleep_score_for_me.v1.ssfm_v1 as ss
import sleep_score_for_me.utils.ssfm_utils as ssu

import adtk
from adtk.detector import GeneralizedESDTestAD
from adtk.visualization import plot

bp_def = dict(delta=(1.5, 4), theta=(4, 8), sigma = (11, 16), beta = (13, 20), low_gamma = (40, 55), high_gamma = (65, 80), omega=(300, 700))

kd_ref = {}
kd_ref['echans'] = [1,2]
kd_ref['fchans']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
kd_ref['analysis_root'] = Path('/Volumes/opto_loc/Data/ACHR_PROJECT_MATERIALS/ACHR_3/ACHR_3-analysis-data')
kd_ref['tank_root'] = Path('/Volumes/opto_loc/Data/ACHR_3/ACHR_3_TANK')

In [2]:
# Will use ACHR_3 Data to start:

pth_x1_control = '/Volumes/opto_loc/Data/ACHR_3/ACHR_3_TANK/ACHR_3-exp1-control'
pth_x1_control_bl = '/Volumes/opto_loc/Data/ACHR_3/ACHR_3_TANK/ACHR_3-exp1-control-bl'
pth_x2 = '/Volumes/opto_loc/Data/ACHR_3/ACHR_3_TANK/ACHR_3-exp2'
pth_x2_bl = '/Volumes/opto_loc/Data/ACHR_3/ACHR_3_TANK/ACHR_3-exp2-bl'

arej = {}

- First goal is to get this working on some data from ACHR_3 Exp2-Baseline, which seems to have some bad artifact in the deep LFP channels, and in the EEG channels

In [4]:
import adtk
from adtk.detector import GeneralizedESDTestAD
from adtk.visualization import plot

In [3]:
arej['e'], arej['espg'] = kd.get_data_spg(pth_x2_bl, store='EEGr', t1=0, t2=10800, channel=kd_ref['echans'])
arej['f'], arej['fspg'] = kd.get_data_spg(pth_x2_bl, store='LFP_', t1=0, t2=10800, channel=kd_ref['fchans'])

read from t=0s to t=11370.42s
Using 1525.8789 Hz as SEV sampling rate for EEGr
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=11370.42s
Using 3051.7578 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [6]:
arej['e.pd'], arej['espg.pd'] = kd.get_data_spg(pth_x2_bl, store='EEGr', t1=0, t2=10800, channel=kd_ref['echans'], pandas=True)
arej['f.pd'], arej['fspg.pd'] = kd.get_data_spg(pth_x2_bl, store='LFP_', t1=0, t2=10800, channel=kd_ref['fchans'], pandas=True)

read from t=0s to t=11370.42s
Using 1525.8789 Hz as SEV sampling rate for EEGr
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=11370.42s
Using 3051.7578 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [23]:
b, s = kp.plot_bp_and_spectro(spg=arej['fspg'], chan=3, hyp=None, bp_def=bp_def, band='delta', fig_size=(20,5))
b.set_ylim(0, 65000)
s.set_ylim(0, 16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 16.0)

In [18]:
b, s = kp.plot_bp_and_spectro(spg=arej['fspg'], chan=9, hyp=None, bp_def=bp_def, band='delta', fig_size=(20,5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
bp9 = kd.get_bp_set2(spg=arej['fspg'].sel(channel=9), bands=bp_def, pandas=True)

In [41]:
f9_delta = bp9['delta']

In [33]:
bp9d = bp9.delta

In [29]:
kd.get_bp_set2?

Signature: kd.get_bp_set2(spg, bands, pandas=False)
Docstring: <no docstring>
File:      ~/github_t2/kd_analysis/kd_analysis/main/utils.py
Type:      function


In [27]:
f9 = arej['f.pd'].xs(9, level='channel')

In [28]:
f9

,LFP_
datetime,
2021-12-14 09:02:21.999999000,-109.428833
2021-12-14 09:02:22.000326680,-114.137085
2021-12-14 09:02:22.000654360,-118.779182
2021-12-14 09:02:22.000982039,-111.455574
2021-12-14 09:02:22.001309720,-99.647324
...,...
2021-12-14 12:02:21.998563631,52.526932
2021-12-14 12:02:21.998891311,47.167984
2021-12-14 12:02:21.999218991,35.126698


In [43]:
esd_ad = GeneralizedESDTestAD(alpha=0.3)
anomalies = esd_ad.fit_detect(f9_delta)
from adtk.visualization import plot
plot(f9_delta, anomaly=anomalies, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
anomalies

datetime
2021-12-14 09:02:23.999993880    False
2021-12-14 09:02:27.000231959    False
2021-12-14 09:02:30.000470039    False
2021-12-14 09:02:33.000708118    False
2021-12-14 09:02:36.000946198    False
                                 ...  
2021-12-14 12:02:06.855651633    False
2021-12-14 12:02:09.855889713    False
2021-12-14 12:02:12.856127792    False
2021-12-14 12:02:15.856365872    False
2021-12-14 12:02:18.856603951    False
Name: delta, Length: 3599, dtype: bool

In [45]:
f9_delta

datetime
2021-12-14 09:02:23.999993880    37108.894531
2021-12-14 09:02:27.000231959    22488.562500
2021-12-14 09:02:30.000470039    20750.445312
2021-12-14 09:02:33.000708118    22664.792969
2021-12-14 09:02:36.000946198     8745.557617
                                     ...     
2021-12-14 12:02:06.855651633    25321.601562
2021-12-14 12:02:09.855889713     8955.240234
2021-12-14 12:02:12.856127792    22142.025391
2021-12-14 12:02:15.856365872    56147.417969
2021-12-14 12:02:18.856603951     6302.315918
Name: delta, Length: 3599, dtype: float32

In [46]:
f9d_df = f9_delta.to_frame(name='delta')

In [47]:
f9d_df['arts'] = anomalies.values

In [72]:
f9d_df.loc[f9d_df.arts==True, 'delta'] = np.NaN

In [73]:
f9d_df 

,delta,arts
datetime,,
2021-12-14 09:02:23.999993880,37108.894531,False
2021-12-14 09:02:27.000231959,22488.562500,False
2021-12-14 09:02:30.000470039,20750.445312,False
2021-12-14 09:02:33.000708118,22664.792969,False
2021-12-14 09:02:36.000946198,8745.557617,False
...,...,...
2021-12-14 12:02:06.855651633,25321.601562,False
2021-12-14 12:02:09.855889713,8955.240234,False
2021-12-14 12:02:12.856127792,22142.025391,False


In [67]:
from scipy.stats import mode
samp = mode(np.diff(f9d_df.index.values)).mode[0]

In [68]:
samp

numpy.timedelta64(3000238080,'ns')

In [74]:
f, ax = plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x=f9d_df.index, y=f9d_df['delta'], ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
f, ax = plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x=sd9.index, y=sd9['delta'], ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
sd9 = get_smoothed_df(f9d_df, col='delta')

In [69]:
def get_smoothed_df(df, col, smoothing_sigma=10):
    df = df.copy()
    period = scipy.stats.mode(np.diff(df.index.values)).mode[0]
    period = period/pd.to_timedelta(1, 's')
    fs = 1/period
    data = df[col]
    smoothed_data = kd.gaussian_smooth(data, sigma=smoothing_sigma, sampling_frequency=fs)
    df[col] = smoothed_data
    return df